## Homework 1
### ECON470, Spring 2026
#### Valerie Hernandez

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

In [2]:
# Read merged data set for 2018
merged_data = pd.read_csv("../../../data/output/merged_plan_service_area.csv", encoding="latin1")

def _not_na_string(s):
    return s.notna() & (s.astype(str) != "NA")

/var/folders/dr/bz36x93s5vs6lnjjq8z60zfm0000gp/T/ipykernel_34917/2862271358.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv("../../../data/output/merged_plan_service_area.csv", encoding="latin1")


In [3]:
# Question 1: Provide a table of the count of plans under each plan type.
plan_type_year1 = (
    merged_data.loc[_not_na_string(merged_data["plan_type_plan"])]
    .groupby("plan_type_plan", dropna=False)
    .size()
    .reset_index(name="Count")
    .sort_values("Count", ascending=False)
)

In [12]:
# Question 2: Remove all special needs plans (SNP), employer group plans (eghp), and all “800-series” plans.
planid_num = pd.to_numeric(merged_data.get("planid"), errors="coerce")
plan_type_year2 = (
    merged_data.loc[
        (merged_data.get("snp") == "No")
        & (merged_data.get("eghp_plan") == "No")
        & ((planid_num < 800) | (planid_num >= 900))
        & _not_na_string(merged_data["plan_type_plan"])
    ]
    .groupby("plan_type_plan", dropna=False)
    .size()
    .reset_index(name="Count")
    .sort_values("Count", ascending=False)
)

In [13]:
# Question 3: Now repeat the same filters from part 2 but also focus only on counties in which plans are approved as per the service area files. 
# First, check if we have an enrollment column
enroll_cols = [c for c in merged_data.columns if "enroll" in c.lower()]
print("Available enrollment columns:", enroll_cols)

# Use the enrollment column if it exists
if enroll_cols:
    enroll_col = enroll_cols[0]
else:
    # If no enrollment column, create a placeholder
    merged_data["enrollment"] = np.nan
    enroll_col = "enrollment"

plan_type_enroll = (
    merged_data.loc[
        (merged_data.get("snp") == "No")
        & (merged_data.get("eghp_plan") == "No")
        & ((pd.to_numeric(merged_data.get("planid"), errors="coerce") < 800)
           | (pd.to_numeric(merged_data.get("planid"), errors="coerce") >= 900))
        & _not_na_string(merged_data["plan_type_plan"])
    ]
    .groupby("plan_type_plan", dropna=False)
    .agg(
        Count=("plan_type_plan", "size"),
        Avg_Enrollments=(enroll_col, "mean"),
    )
    .reset_index()
    .sort_values("Count", ascending=False)
)

Available enrollment columns: ['enrollment']


### 1. Provide a table of plan types in 2018.

#### Display Table 1

In [14]:
print("Table 1: All plan types in 2018")
print(plan_type_year1)

Table 1: All plan types in 2018
                      plan_type_plan   Count
2                          Local PPO  902685
1                         HMO/HMOPOS  425617
7                       Regional PPO   15920
6                               PFFS   15337
0                          1876 Cost    5418
5                      National PACE     902
4  Medicare-Medicaid Plan HMO/HMOPOS     418
3                                MSA     238


### 2. Remove all special needs plans (SNP), employer group plans (eghp), and all "800-series" plans. Provide an updated table after making these exclusions.

#### Display Table 2
##### Here, we remove the relevant plans by applying the relevant filters described above. The resulting plan types with these exclusions are shown in the resulting table. 

In [15]:
print("Table 2: Revised plan types in 2018")
print(plan_type_year2)

Table 2: Revised plan types in 2018
                      plan_type_plan  Count
2                          Local PPO  38574
1                         HMO/HMOPOS  33236
7                       Regional PPO   6797
0                          1876 Cost   4872
6                               PFFS   2726
5                      National PACE    902
4  Medicare-Medicaid Plan HMO/HMOPOS    418
3                                MSA    183


### 3. Now calculate the average enrollments by plan type in 2018, using the same exclusions as above.

#### Display Table 3

#### To do this, we had to calculate the number of plan–county observations and the mean enrollment across the counties in which each plan type operates. The resulting table is shown.

In [18]:
print("Table 3: Enrollments by plan type in 2018")
tbl = plan_type_enroll.copy()
if "Avg_Enrollments" in tbl.columns:
    tbl["Avg_Enrollments"] = tbl["Avg_Enrollments"].round(0)
print(tbl)

Table 3: Enrollments by plan type in 2018
                      plan_type_plan  Count  Avg_Enrollments
2                          Local PPO  38574            338.0
1                         HMO/HMOPOS  33236            763.0
7                       Regional PPO   6797            187.0
0                          1876 Cost   4872            249.0
6                               PFFS   2726             90.0
5                      National PACE    902            147.0
4  Medicare-Medicaid Plan HMO/HMOPOS    418           1022.0
3                                MSA    183             60.0


In [8]:
# Check what values are in the snp column
print("SNP values:")
print(merged_data["snp"].value_counts(dropna=False))

SNP values:
snp
No     1350940
Yes      15595
Name: count, dtype: int64


In [10]:
# Check planid distribution
planid_num = pd.to_numeric(merged_data.get("planid"), errors="coerce")
print("\nPlanid stats:")
print(planid_num.describe())
print("\n800-series plans (planid between 800-899):")
print(((planid_num >= 800) & (planid_num < 900)).sum())


Planid stats:
count    1.366535e+06
mean     7.496710e+02
std      2.041579e+02
min      1.000000e+00
25%      8.020000e+02
50%      8.040000e+02
75%      8.100000e+02
max      9.990000e+02
Name: planid, dtype: float64

800-series plans (planid between 800-899):
1263242


In [11]:
# See all columns in merged_data
print("All columns in merged_data:")
print(merged_data.columns.tolist())

All columns in merged_data:
['contractid', 'planid', 'fips', 'year', 'state_plan', 'county_plan', 'org_type_plan', 'plan_type_plan', 'partd', 'snp', 'eghp_plan', 'org_name_plan', 'org_marketing_name', 'plan_name', 'parent_org', 'contract_date', 'enrollment', 'state_sa', 'county_sa', 'org_name_sa', 'org_type_sa', 'plan_type_sa', 'partial', 'eghp_sa', 'ssa', 'notes']
